# Darcy equation

In this tutorial we present how to solve an evolutionary Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Let $(0,T)$ with $10=T>0$ be the overall simulation period. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = {- \rho g \nabla y}\\
p_t + \nabla \cdot {q} = f
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \times (0,T] \qquad p = \rho g \text{ on } \partial_{bottom} \Omega \times (0,T] \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \times (0,T] $$
and initial conditions:
$$ p|_{t=0} = (1-y) \rho g \text{ in } \Omega \qquad q|_{t=0} = 0 \text{ in } \Omega $$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
%load_ext Cython

In [2]:
import shutil
import os

import numpy as np
import scipy.sparse as sps
import scipy.integrate as integrate

import porepy as pp
import pygeon as pg

import sympy as sp

from math import ceil, floor, log10, exp

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


For a manual import:

import sys
sys.path.append('/path/to/repository')

In [3]:
from utilities.assembly_utilities import find_ordering, experimental_local_A, experimental_local_Mh, transoform_nodal_func_to_physical_element
from richards.model_params import Model_Data

### Parameters

In [4]:
# Set the maximum number of iterations of the non-linear solver
K = 50

# L-scheme parameter
L = 3.501e-2 #0.1

# Set the mesh refinment
N = 10

# Set the number of steps (excluding the initial condition)
num_steps = 9

quad_order = 3

# Simulation time length
T = 9/48

# Time switch conditions (for the boundary condition)
dt_D = 3/48

# Fluid density
rho = 1000

# Relative and absolute tolerances for the non-linear solver
abs_tol = 1e-10
rel_tol = 1e-5

# Domain tolerance
domain_tolerance = 1 / (10 * N)

# Output directory
output_directory = 'hard_coupling'

In [5]:
def initial_h(x): return 1

In [6]:
# Van Genuchten model parameters ( relative permeability model )
model_data = Model_Data(theta_r=0.131, theta_s=0.396, alpha=0.423, n=2.06, K_s=4.96e-2, T=T, num_steps=num_steps)

In [7]:
# Time step
dt   = (T-0)/num_steps

### Mesh

In [8]:
bottom_grid =   pp.StructuredTriangleGrid([2*N, N], [2, 1])
bottom_grid.compute_geometry()

In [9]:
top_grid    = pp.StructuredTriangleGrid([2*N, 2*N], [2, 1])
top_grid.compute_geometry()

In [10]:
boundary_grid, boundary_face_map, boundary_node_map  = pp.partition.extract_subgrid(bottom_grid, 
                                                                                    bottom_grid.face_centers[1, :] == 1, 
                                                                                    faces=True)

### Discretization

In [11]:
key = "flow"

In [12]:
top_field = pg.Lagrange1(key)
top_data = {}

top_dof =  top_field.ndof( top_grid )

In [13]:
bottom_field = pg.Lagrange1(key)
bottom_data = {}

bottom_dof = bottom_field.ndof( bottom_grid )

In [14]:
boundary_field = pg.Lagrange1(key)
boundary_data = {}

boundary_dof = boundary_field.ndof( boundary_grid )

### Restrictor Matrices

In [15]:
bottom_delete_boundary_dof = np.zeros( shape=(bottom_dof - boundary_dof, bottom_dof) )
bottom_delete_boundary_dof[np.arange(bottom_dof - boundary_dof), bottom_grid.nodes[1, :] < 1] = 1
bottom_delete_boundary_dof = sps.csr_matrix(bottom_delete_boundary_dof)

In [16]:
bottom_isolate_boundary_dof = np.zeros(shape=(boundary_dof, bottom_dof))
bottom_isolate_boundary_dof[np.arange(boundary_dof), bottom_grid.nodes[1, :] == 1] = 1
bottom_isolate_boundary_dof = sps.csr_matrix(bottom_isolate_boundary_dof)

In [17]:
top_delete_boundary_dof = np.zeros( shape=(top_dof - boundary_dof, top_dof) )
top_delete_boundary_dof[np.arange(top_dof - boundary_dof), top_grid.nodes[1, :] > 0] = 1
top_delete_boundary_dof = sps.csr_matrix(top_delete_boundary_dof)

In [18]:
top_isolate_boundary_dof = np.zeros(shape=(boundary_dof, top_dof))
top_isolate_boundary_dof[np.arange(boundary_dof), top_grid.nodes[1, :] == 0] = 1
top_isolate_boundary_dof = sps.csr_matrix(top_isolate_boundary_dof)

### BCs

In [19]:
bot_dirichlet_flag  = np.array(bottom_delete_boundary_dof @ np.logical_and( bottom_grid.nodes[0, :] == 2, bottom_grid.nodes[1, :] <= 1 ), dtype=bool)
bot_dirichlet_value = np.array(bot_dirichlet_flag, dtype=float)

In [20]:
top_dirichlet_flag  = np.array(top_delete_boundary_dof @ np.logical_and( top_grid.nodes[1, :] == 1, top_grid.nodes[0, :] <= 1 ), dtype=bool)
top_dirichlet_value = lambda t: np.array(top_dirichlet_flag, dtype=float) * min( 3.2, 1 + 2.2 * t / dt_D )

In [21]:
dirichlet_flag  = np.hstack(( bot_dirichlet_flag, np.zeros(shape=boundary_dof, dtype=bool), top_dirichlet_flag ))
dirichlet_value = lambda t: np.hstack(( bot_dirichlet_value, np.zeros(shape=boundary_dof), top_dirichlet_value(t)))

### Matrix Assembly

##### Projection matrices

In [22]:
top_proj    = top_field.eval_at_cell_centers( top_grid )
bottom_proj = bottom_field.eval_at_cell_centers( bottom_grid )

##### Stifness

In [23]:
def global_stifness(h_dofs):
    
    size = np.power(bottom_grid.dim + 1, 2) * bottom_grid.num_cells + np.power(top_grid.dim + 1, 2) * top_grid.num_cells
    rows_I = np.empty(size, dtype=int)
    cols_J = np.empty(size, dtype=int)
    data_IJ = np.empty(size)
    idx = 0

    psi = h_dofs - (1 + 2 * top_grid.nodes[1,:])

    for subdomain, base in zip([bottom_grid, top_grid], [0, boundary_grid.num_nodes * N]):
        _, _, _, _, _, node_coords = pp.map_geometry.map_grid(subdomain)

        # Allocate the data to store matrix entries, that's the most efficient
        # way to create a sparse matrix.

        cell_nodes = subdomain.cell_nodes()

        for c in np.arange(subdomain.num_cells):
            # For the current cell retrieve its nodes
            loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

            nodes_loc = cell_nodes.indices[loc]
            coord_loc = node_coords[:, nodes_loc]

            # Compute the stiff-H1 local matrix

            if subdomain == bottom_grid:
                A = experimental_local_A(coord_loc, 
                                         lambda x,y: model_data.K_s * np.eye(2), 
                                         quad_order)
            else:
                h_func = transoform_nodal_func_to_physical_element(h_dofs[nodes_loc], coord_loc)
                A = experimental_local_A(
                    coord_loc,
                    lambda x,y: model_data.hydraulic_conductivity_coefficient(h_func(x,y), 1+2*y) * np.array([[2, 0], [0, 1/2]]),
                    quad_order
                )

            # Save values for stiff-H1 local matrix in the global structure
            cols = base + np.tile(nodes_loc, (nodes_loc.size, 1))

            loc_idx = slice(idx, idx + cols.size)
            rows_I[loc_idx] = cols.T.ravel()
            cols_J[loc_idx] = cols.ravel()
            data_IJ[loc_idx] = A.ravel()
            idx += cols.size

    # Construct the global matrices
    return sps.csc_matrix((data_IJ, (rows_I, cols_J)))

##### Mass

In [24]:
def global_mass():
    
    size = np.power(bottom_grid.dim + 1, 2) * bottom_grid.num_cells + np.power(top_grid.dim + 1, 2) * top_grid.num_cells
    rows_I = np.empty(size, dtype=int)
    cols_J = np.empty(size, dtype=int)
    data_IJ = np.empty(size)
    idx = 0

    for subdomain, base, mult in zip([bottom_grid, top_grid], [0, boundary_grid.num_nodes * N], [1,2]):
        _, _, _, _, _, node_coords = pp.map_geometry.map_grid(subdomain)

        # Allocate the data to store matrix entries, that's the most efficient
        # way to create a sparse matrix.

        cell_nodes = subdomain.cell_nodes()

        for c in np.arange(subdomain.num_cells):
            # For the current cell retrieve its nodes
            loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

            nodes_loc = cell_nodes.indices[loc]
            coord_loc = node_coords[:, nodes_loc]

            # Compute the stiff-H1 local matrix
            A = experimental_local_Mh(coord_loc, lambda x,y: 1, quad_order) * mult

            # Save values for stiff-H1 local matrix in the global structure
            cols = base + np.tile(nodes_loc, (nodes_loc.size, 1))

            loc_idx = slice(idx, idx + cols.size)
            rows_I[loc_idx] = cols.T.ravel()
            cols_J[loc_idx] = cols.ravel()
            data_IJ[loc_idx] = A.ravel()
            idx += cols.size

    # Construct the global matrices
    return sps.csc_matrix((data_IJ, (rows_I, cols_J)))

### Solving stage

In [25]:
# assemble initial solution
initial_solution = np.zeros(top_dof + bottom_dof - boundary_dof)

In [26]:
bottom_mask = np.zeros_like(initial_solution, dtype=bool)
bottom_mask[ : (bottom_dof) ] = True

internal_bottom_mask = np.zeros_like(bottom_mask, dtype=bool)
internal_bottom_mask[ : (bottom_dof - boundary_dof)] = True

top_mask = np.zeros_like(initial_solution, dtype=bool)
top_mask[(bottom_dof - boundary_dof) : ] = True

internal_top_mask = np.zeros_like(bottom_mask, dtype=bool)
internal_top_mask[ bottom_dof : ] = True

boundary_mask = np.zeros_like(bottom_mask, dtype=bool)
boundary_mask[(bottom_dof - boundary_dof) : bottom_dof] = True

In [27]:
initial_solution[top_mask]    =    top_field.interpolate(   top_grid, lambda x: 1)
initial_solution[bottom_mask] = bottom_field.interpolate(bottom_grid, lambda x: 1)

In [28]:
top_saver    = pp.Exporter(top_grid,    "top_sol", folder_name=output_directory)
bottom_saver = pp.Exporter(bottom_grid, "bottom_sol", folder_name=output_directory)

def save_step(current_sol, step):
    ins = list()

    ins.append((top_grid, "cell_h", top_proj @ current_sol[top_mask]))
    ins.append((top_grid, "cell_p", top_proj @ (current_sol[top_mask] - 1 - top_grid.nodes[1, :] * 2)))
    top_saver.write_vtu(ins, time_step=step)

    ins = list()

    ins.append((bottom_grid, "cell_h", bottom_proj @ current_sol[bottom_mask]))
    ins.append((bottom_grid, "cell_p", bottom_proj @ (current_sol[bottom_mask] - bottom_grid.nodes[1, :])))
    bottom_saver.write_vtu(ins, time_step=step)

In [29]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [30]:
# solve the problem

sol = [initial_solution]

t = 0

save_step(sol[-1], 0)

In [31]:
csv_base = os.path.join(output_directory, 'csv')

if os.path.exists(csv_base):
    shutil.rmtree(csv_base)
    
os.mkdir(csv_base)

In [32]:
glob_mass = global_mass()
top_mass = glob_mass[top_mask, :][:, top_mask]
bottom_mass = glob_mass[bottom_mask, :][:, bottom_mask]

In [34]:
# Time Loop
for step in range(1, ceil(T/dt) + 1):
    current_time = step * dt
    print('Time ' + str(round(current_time, 5)))
    
    time_rhs = np.zeros_like(sol[-1])

    time_rhs[top_mask] += top_mass @ model_data.theta(sol[-1][top_mask], 1 + top_grid.nodes[1,:] * 2) / dt

    prev = sol[-1]

    for k in range(K):
        rhs = time_rhs.copy()

        rhs[top_mask] += L * top_mass @ prev[top_mask] / dt
        rhs[top_mask] -= top_mass @ model_data.theta(prev[top_mask], 1 + top_grid.nodes[1,:] * 2) / dt

        global_stif = global_stifness( prev[top_mask] )# - (1 + top_grid.nodes[1,:] * 2) )

        bottom_mat = global_stif[bottom_mask, :][:, bottom_mask] #+ L / dt * bottom_mass
        top_mat    = global_stif[top_mask, :][:, top_mask] + L / dt * top_mass
        mid_mat = global_stif[boundary_mask, :][:, boundary_mask] + L / dt * glob_mass[boundary_mask, :][:, boundary_mask]
        
        spp = sps.bmat([[    bottom_delete_boundary_dof @ bottom_mat @ bottom_delete_boundary_dof.T, bottom_delete_boundary_dof @ bottom_mat @ bottom_isolate_boundary_dof.T,                                                             None],
                        [ bottom_isolate_boundary_dof @ bottom_mat.T @ bottom_delete_boundary_dof.T,                                                                 mid_mat, top_isolate_boundary_dof @ top_mat.T @ top_delete_boundary_dof.T],
                        [                                                                      None,          top_delete_boundary_dof @ top_mat @ top_isolate_boundary_dof.T,    top_delete_boundary_dof @ top_mat @ top_delete_boundary_dof.T]], format = 'csc')
        
        ls = pg.LinearSystem(spp, rhs)

        #print(spp, rhs)

        ls.flag_ess_bc(dirichlet_flag, dirichlet_value(current_time))

        current = ls.solve()

        # Check if we have reached convergence
        rel_err_psi  = np.sqrt( (current - prev).T @ glob_mass @ (current - prev) )
        abs_err_prev = np.sqrt( prev.T @ glob_mass @ prev )

        # Log message with error and current iteration
        print('Iteration #' + format(k+1, '0' + str(ceil(log10(K)) + 1) + 'd') + ', error L2 relative psi: ' 
              + format(rel_err_psi, str(5 + ceil(log10(1 / abs_tol)) + 4) + '.' + str(ceil(log10(1 / abs_tol)) + 4) + 'f') )
        
        if rel_err_psi > abs_tol + rel_tol * abs_err_prev:
            prev = current.copy()
        else:
            break

    sol.append( current )

    save_step(sol[-1], step)
    print('')    

Time 0.02083


Iteration #001, error L2 relative psi:    0.14776180904585


<lambdifygenerated-1>:2: RuntimeWarning: invalid value encountered in power
  return select([less(h, z),True], [0.265*(0.1699265174169*(-h + z)**2.06 + 1)**(-0.514563106796116) + 0.131,0.396], default=nan)


Iteration #002, error L2 relative psi:    0.01610614825236
Iteration #003, error L2 relative psi:    0.00070124722068
Iteration #004, error L2 relative psi:    0.00009530562462
Iteration #005, error L2 relative psi:    0.00001854194896

Time 0.04167
Iteration #001, error L2 relative psi:    0.20128318736975
Iteration #002, error L2 relative psi:    0.05279049239485
Iteration #003, error L2 relative psi:    0.00552610576146
Iteration #004, error L2 relative psi:    0.00047711407817
Iteration #005, error L2 relative psi:    0.00005273988565


KeyboardInterrupt: 

# Time Loop
for step in range(1, ceil(T/dt) + 1):
    current_time = step * dt
    print('Time ' + str(round(current_time, 5)))
    
    time_rhs = np.zeros_like(sol[-1])

    time_rhs[top_mask] += top_mass @ theta(sol[-1][top_mask] - (1 + top_grid.nodes[1,:] * 2)) / dt
    time_rhs[internal_bottom_mask] += bottom_delete_boundary_dof @ bottom_mass @ theta(sol[-1][bottom_mask] - bottom_grid.nodes[1,:]) / dt

    time_rhs[boundary_mask] += glob_mass[boundary_mask, :][:, internal_bottom_mask] @ theta(sol[-1][internal_bottom_mask] - bottom_grid.nodes[1,:(2*N+1)*N]) / dt

    prev = sol[-1]

    for k in range(K):
        rhs = time_rhs.copy()

        rhs[top_mask] += L * top_mass @ prev[top_mask] / dt
        rhs[top_mask] -= top_mass @ theta(prev[top_mask] - (1 + top_grid.nodes[1,:] * 2)) / dt

        rhs[internal_bottom_mask] += L * bottom_delete_boundary_dof @ bottom_mass @ prev[bottom_mask] / dt
        rhs[internal_bottom_mask] -= bottom_delete_boundary_dof @ bottom_mass @ theta(prev[bottom_mask] - bottom_grid.nodes[1, :]) / dt

        rhs[boundary_mask] += L * glob_mass[boundary_mask, :][:, internal_bottom_mask] @ prev[internal_bottom_mask] / dt
        rhs[boundary_mask] -= glob_mass[boundary_mask, :][:, internal_bottom_mask] @ theta(prev[internal_bottom_mask] - bottom_grid.nodes[1,:(2*N+1)*N]) / dt

        global_stif = global_stifness( prev[top_mask] )#- (1 + top_grid.nodes[1,:] * 2) )

        bottom_mat = global_stif[bottom_mask, :][:, bottom_mask] + L / dt * bottom_mass
        top_mat    = global_stif[top_mask, :][:, top_mask] + L / dt * top_mass
        mid_mat = global_stif[boundary_mask, :][:, boundary_mask] + L / dt * glob_mass[boundary_mask, :][:, boundary_mask]
        
        spp = sps.bmat([[    bottom_delete_boundary_dof @ bottom_mat @ bottom_delete_boundary_dof.T, bottom_delete_boundary_dof @ bottom_mat @ bottom_isolate_boundary_dof.T,                                                             None],
                        [ bottom_isolate_boundary_dof @ bottom_mat.T @ bottom_delete_boundary_dof.T,                                                                 mid_mat, top_isolate_boundary_dof @ top_mat.T @ top_delete_boundary_dof.T],
                        [                                                                      None,          top_delete_boundary_dof @ top_mat @ top_isolate_boundary_dof.T,    top_delete_boundary_dof @ top_mat @ top_delete_boundary_dof.T]], format = 'csc')
        
        ls = pg.LinearSystem(spp, rhs)

        ls.flag_ess_bc(dirichlet_flag, dirichlet_value(current_time))

        current = ls.solve()

        # Check if we have reached convergence
        rel_err_psi  = np.sqrt( (current - prev).T @ glob_mass @ (current - prev) )
        abs_err_prev = np.sqrt( prev.T @ glob_mass @ prev )

        # Log message with error and current iteration
        print('Iteration #' + format(k+1, '0' + str(ceil(log10(K)) + 1) + 'd') + ', error L2 relative psi: ' 
              + format(rel_err_psi, str(5 + ceil(log10(1 / abs_tol)) + 4) + '.' + str(ceil(log10(1 / abs_tol)) + 4) + 'f') )
        
        if rel_err_psi > abs_tol + rel_tol * abs_err_prev:
            prev = current.copy()
        else:
            break

    sol.append( current )

    save_step(sol[-1], step)
    print('')    

#### Full Richard

# Time Loop
for step in range(1, ceil(T/dt) + 1):
    current_time = step * dt
    print('Time ' + str(round(current_time, 5)))
    
    time_rhs = np.zeros_like(sol[-1])

    time_rhs[internal_top_mask] += top_delete_boundary_dof @ top_mass @ theta(sol[-1][top_mask] - (1 + top_grid.nodes[1,:] * 2)) / dt
    time_rhs[internal_bottom_mask] += bottom_delete_boundary_dof @ bottom_mass @ theta(sol[-1][bottom_mask] - bottom_grid.nodes[1,:]) / dt

    time_rhs[boundary_mask] += glob_mass[boundary_mask, :][:, bottom_mask] @ theta(sol[-1][bottom_mask] - bottom_grid.nodes[1,:]) / dt
    time_rhs[boundary_mask] += glob_mass[boundary_mask, :][:, top_mask] @ theta(sol[-1][top_mask] - (1 + top_grid.nodes[1,:] * 2)) / dt
    time_rhs[boundary_mask] -= glob_mass[boundary_mask, :][:, boundary_mask] @ theta(sol[-1][boundary_mask] - 1) / dt

    prev = sol[-1]

    for k in range(K):
        rhs = time_rhs.copy()

        rhs[internal_top_mask] += L * top_delete_boundary_dof @ top_mass @ prev[top_mask] / dt
        rhs[internal_top_mask] -= top_delete_boundary_dof @ top_mass @ theta(prev[top_mask] - (1 + top_grid.nodes[1,:] * 2)) / dt

        rhs[internal_bottom_mask] += L * bottom_delete_boundary_dof @ bottom_mass @ prev[bottom_mask] / dt
        rhs[internal_bottom_mask] -= bottom_delete_boundary_dof @ bottom_mass @ theta(prev[bottom_mask] - bottom_grid.nodes[1, :]) / dt

        rhs[boundary_mask] += L * glob_mass[boundary_mask, :][:, bottom_mask] @ prev[bottom_mask] / dt
        rhs[boundary_mask] += L * glob_mass[boundary_mask, :][:, top_mask] @ prev[top_mask] / dt
        rhs[boundary_mask] -= L * glob_mass[boundary_mask, :][:, boundary_mask] @ prev[boundary_mask] / dt

        rhs[boundary_mask] -= glob_mass[boundary_mask, :][:, bottom_mask] @ theta(prev[bottom_mask] - bottom_grid.nodes[1,:]) / dt
        rhs[boundary_mask] -= glob_mass[boundary_mask, :][:, top_mask] @ theta(prev[top_mask] - (1 + top_grid.nodes[1,:] * 2)) / dt
        rhs[boundary_mask] += glob_mass[boundary_mask, :][:, boundary_mask] @ theta(prev[boundary_mask] - 1) / dt

        global_stif = global_stifness( prev[top_mask] - (1 + top_grid.nodes[1,:] * 2) )

        bottom_mat = global_stif[bottom_mask, :][:, bottom_mask] + L / dt * bottom_mass
        top_mat    = global_stif[top_mask, :][:, top_mask] + L / dt * top_mass
        mid_mat = global_stif[boundary_mask, :][:, boundary_mask] + L / dt * glob_mass[boundary_mask, :][:, boundary_mask]
        
        spp = sps.bmat([[    bottom_delete_boundary_dof @ bottom_mat @ bottom_delete_boundary_dof.T, bottom_delete_boundary_dof @ bottom_mat @ bottom_isolate_boundary_dof.T,                                                             None],
                        [ bottom_isolate_boundary_dof @ bottom_mat.T @ bottom_delete_boundary_dof.T,                                                                 mid_mat, top_isolate_boundary_dof @ top_mat.T @ top_delete_boundary_dof.T],
                        [                                                                      None,          top_delete_boundary_dof @ top_mat @ top_isolate_boundary_dof.T,    top_delete_boundary_dof @ top_mat @ top_delete_boundary_dof.T]], format = 'csc')
        
        ls = pg.LinearSystem(spp, rhs)

        print(spp, rhs)

        ls.flag_ess_bc(dirichlet_flag, dirichlet_value(current_time))

        current = ls.solve()

        # Check if we have reached convergence
        rel_err_psi  = np.sqrt( (current - prev).T @ glob_mass @ (current - prev) )
        abs_err_prev = np.sqrt( prev.T @ glob_mass @ prev )

        # Log message with error and current iteration
        print('Iteration #' + format(k+1, '0' + str(ceil(log10(K)) + 1) + 'd') + ', error L2 relative psi: ' 
              + format(rel_err_psi, str(5 + ceil(log10(1 / abs_tol)) + 4) + '.' + str(ceil(log10(1 / abs_tol)) + 4) + 'f') )
        
        if rel_err_psi > abs_tol + rel_tol * abs_err_prev:
            prev = current.copy()
        else:
            break

    sol.append( current )

    save_step(sol[-1], step)
    print('')    